In [1]:
import sys

sys.path.append("..")

In [2]:
from config import config
from config import config_indicators

from preprocessor.findata_preprocessor import FinancialDataPreprocessor
from visualiser.findata_visualiser import FinancialDataVisualiser
from environments.env_finrl import FinRLTradingEnv
from agents.agent_finrl import FinRLAgent

In [3]:
import os

if not os.path.exists("../" + config.MODELS_DIR):
    os.makedirs("../" + config.MODELS_DIR)

if not os.path.exists("../" + config.RESULTS_DIR):
    os.makedirs("../" + config.RESULTS_DIR)

In [4]:
finpreprocessor = FinancialDataPreprocessor(config.START_DATE, config.END_DATE)
train_data, trade_data = finpreprocessor.load_train_test_data(
    "../" + config.DATA_DIR, config.TEST_NAME
)

In [5]:
finvisualiser = FinancialDataVisualiser()
finvisualiser.plot_train_test_close_prices(
    train_data, trade_data, "../" + config.PLOT_DIR, config.TEST_NAME
)

/Users/ingridperez/Documents/GitHub Repositiories/xdl-portfolio/examples/../visualiser/findata_visualiser.py:219: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [6]:
environment = FinRLTradingEnv(
    train_data, trade_data, list(config_indicators.TECHNICAL_INDICATORS.keys())
)

Environment successfully created with 
	Stock dimension: 5 
	State space: 91


In [7]:
env_train = environment.get_train_env()

In [8]:
agent = FinRLAgent(env_train)
trained_a2c = agent.train_model("a2c", logger_dir="../" + config.RESULTS_DIR)

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Logging to ../results/a2c
----------------------------------------
| time/                 |              |
|    fps                | 710          |
|    iterations         | 100          |
|    time_elapsed       | 0            |
|    total_timesteps    | 500          |
| train/                |              |
|    entropy_loss       | -7.14        |
|    explained_variance | -1.7         |
|    learning_rate      | 0.0007       |
|    n_updates          | 99           |
|    policy_loss        | 10.1         |
|    reward             | 0.12546048   |
|    reward_max         | 0.40224797   |
|    reward_mean        | -0.026924636 |
|    reward_min         | -0.6772546   |
|    std                | 1.01         |
|    value_loss         | 3.16         |
----------------------------------------
----------------------------------------
| time/                 |              |
|    fps                | 726          |
|    iteration

In [9]:
agent.save_model("../" + config.MODELS_DIR, config.TEST_NAME + "_a2c")

In [10]:
trained_a2c = agent.load_model(
    "a2c", "../" + config.MODELS_DIR, config.TEST_NAME + "_a2c"
)

In [11]:
gym_env, (env_trade, obs_trade) = environment.get_trade_env()

In [12]:
df_account, df_actions = agent.test_model(
    gym_env,
)

hit end!
